#  Tabular Playground Series - Jan 2021

it's hard to analyze this data but im gonna try

In [ ]:
import numpy as np  
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm
from sklearn.model_selection import train_test_split
import xgboost as xgb
from lightgbm import LGBMRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from scipy import stats
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')


In [ ]:
train=pd.read_csv('../input/tabular-playground-series-jan-2021/train.csv')
test=pd.read_csv('../input/tabular-playground-series-jan-2021/test.csv')
test_id=test['id']

In [ ]:
train.head()

columns just cont1:14

In [ ]:
list(train.columns)

# Overview to target

In [ ]:
train['target'].describe()

In [ ]:
plt.figure(figsize=(10,7))
sns.distplot(train['target'], color= "R" )
plt.title('Distribution of Target', fontsize=18)
plt.show()
print("Skewness: %f" % train['target'].skew())
print("Kurtosis: %f" % train['target'].kurt())

In [ ]:
plt.figure(figsize=(10,7))
ax =sns.boxplot(x=train["target"],color='R')
plt.title('Distribution of Target', fontsize=18)
plt.show()

In [ ]:
plt.figure(figsize=(10,7))
res = stats.probplot(train['target'], plot=plt)
plt.show()

# Correlation Table

In [ ]:
f, ax = plt.subplots(figsize=(18, 10))
corrmat = train.corr().sort_values(by='target',ascending=False).T
mask=np.triu(np.ones_like(train.corr(), dtype=bool))
sns.heatmap(corrmat, vmax=.2,annot=True,mask=mask,cmap="Reds")
plt.title('Correlation Table')
plt.show()

Top column affecting target (con7 : cont2) both = 0.067 

# Overview to all Data

In [ ]:
cols = corrmat.nlargest(6, 'target')['target'].index
sns.pairplot(train[cols], size = 2.5)
plt.show()

In [ ]:
f, ax = plt.subplots(nrows=4, ncols=4, figsize=(18, 12))
sns.kdeplot(train.iloc[:,1], shade=True,ax=ax[0,0],color='red')
sns.kdeplot(train.iloc[:,2], shade=True,ax=ax[0,1],color='red')
sns.kdeplot(train.iloc[:,3], shade=True,ax=ax[0,2],color='red')
sns.kdeplot(train.iloc[:,4], shade=True,ax=ax[0,3],color='red')

sns.kdeplot(train.iloc[:,5], shade=True,ax=ax[1,0],color='red')
sns.kdeplot(train.iloc[:,6], shade=True,ax=ax[1,1],color='red')
sns.kdeplot(train.iloc[:,7], shade=True,ax=ax[1,2],color='red')
sns.kdeplot(train.iloc[:,8], shade=True,ax=ax[1,3],color='red')

sns.kdeplot(train.iloc[:,9], shade=True,ax=ax[2,0],color='red')
sns.kdeplot(train.iloc[:,10], shade=True,ax=ax[2,1],color='red')
sns.kdeplot(train.iloc[:,11], shade=True,ax=ax[2,2],color='red')
sns.kdeplot(train.iloc[:,12], shade=True,ax=ax[2,3],color='red')

sns.kdeplot(train.iloc[:,13], shade=True,ax=ax[3,0],color='red')
sns.kdeplot(train.iloc[:,14], shade=True,ax=ax[3,1],color='red')

f.delaxes(ax[3, 2])
f.delaxes(ax[3, 3])
plt.tight_layout()
plt.show()


# RegPlot

In [ ]:
fig, ((ax1, ax2), (ax3, ax4))= plt.subplots(nrows=2, ncols=2, figsize=(14,8))
var1 = cols[1]
data = pd.concat([train['target'], train[var1]], axis=1)
sns.regplot(y=var1, x='target', data=data, fit_reg=True, ax=ax1)


var2 = cols[2]
data = pd.concat([train['target'], train[var2]], axis=1)
sns.regplot(y=var2, x='target', data=data, fit_reg=True, ax=ax2,color='red', marker='s')

var3 =  cols[3]
data = pd.concat([train['target'], train[var3]], axis=1)
sns.regplot(y=var3, x='target', data=data, fit_reg=True, ax=ax3,color='orange', marker='^')

var4 = cols[4]
data = pd.concat([train['target'], train[var4]], axis=1)
sns.regplot(y=var4, x='target', data=data, fit_reg=True, ax=ax4,color='green', marker='+')

plt.show()

# Machine Learning

In [ ]:
test=test.drop('id',axis=1)
X=train.iloc[:,1:-1]
y=train.iloc[:,-1]
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=.1,random_state=42,shuffle=True)

In [ ]:
regressor = xgb.XGBRegressor(learning_rate=0.02,max_depth=10,
                            n_estimators=7000,reg_alpha=0.9,
                            reg_lambda=0.9,subsample=0.9,seed=42
                            ,metric_period=100,silent=1)
regressor.fit(X_train, y_train, early_stopping_rounds=6, eval_set=[(X_test, y_test)], verbose=1)

In [ ]:
pred_1 = regressor.predict(test)

In [ ]:
X = train.drop(['id','target'], axis=1)
Xtest = test
y = train['target']

t= int(len(X)*0.9)
X_train, X_test = X.iloc[:t], X.iloc[t:]
y_train, y_test = y.iloc[:t], y.iloc[t:]

In [ ]:
params = {'objective': 'regression',
 'metric': 'rmse',
 'verbosity': -1,
 'boosting_type': 'gbdt',
 'feature_pre_filter': False,
 'learning_rate': 0.005,
 'num_leaves': 102,
 'min_child_samples': 20,
 'sub_feature': 0.4,
 'sub_row': 1,
 'subsample_freq': 0,
 'lambda_l1': 4.6,
 'lambda_l2': 1.9}

In [ ]:
N_FOLDS = 5

kf = KFold(n_splits = N_FOLDS)
oof = np.zeros(len(y))
oof_vanilla = np.zeros(len(y))
preds = np.zeros(len(Xtest))
params['learning_rate'] = 0.005
params['num_iterations'] = 5000
for train_ind, test_ind in tqdm(kf.split(X)):
    X_train = X.iloc[train_ind]
    X_test = X.iloc[test_ind]
    y_train = y.iloc[train_ind]
    y_test = y.iloc[test_ind]

    model = LGBMRegressor(**params)
    vanilla_model = LGBMRegressor()
    
    model.fit(X_train, y_train, eval_set = ((X_test,y_test)), early_stopping_rounds = 50, verbose = 0)
    vanilla_model.fit(X_train, y_train)
    p = model.predict(X_test)
    p_vanilla = vanilla_model.predict(X_test)
    oof[test_ind] = p
    oof_vanilla[test_ind] = p_vanilla
    
    preds += model.predict(Xtest)/N_FOLDS
    
print(f'mean square error (vanilla model): {np.round(mean_squared_error(y, oof_vanilla, squared=False),5)}')    
print(f'mean square error  (with optuna tuning): {np.round(mean_squared_error(y, oof, squared=False),5)}')

In [ ]:
results = [y + -x*0.01 for x, y in zip(pred_1, preds)]

In [ ]:
result = pd.DataFrame({"id":test_id, "target":results})
result.to_csv('submission.csv', index=False)

# See Ya